<img src="https://github.com/ray-project/ray/raw/master/doc/source/images/ray_header_logo.png">

Ray provides a simple, universal API for building distributed applications.

  * Providing simple primitives for building and running distributed applications.
  * Enabling end users to parallelize single machine code, with little to zero code changes.
  * Including a large ecosystem of applications, libraries, and tools on top of the core Ray to enable complex applications.

https://docs.ray.io/en/latest/using-ray.html

In [3]:
#!pip install -U "ray[default]"

Ray programs are able to parallelize and distribute by leveraging an underlying Ray runtime. The Ray runtime consists of multiple services/processes started in the background for communication, data transfer, scheduling, and more. The Ray runtime can be started on a laptop, a single server, or multiple servers.

We start it here with `ray.init()` and stop the runtime with `ray.shutdown()`. Calling `ray.init()` (without any address args) starts a Ray runtime on your laptop/machine. 

In [18]:
import ray

# Start Ray. If you're connecting to an existing cluster, you would use
# ray.init(address=<cluster-address>) instead.
ray.init(num_cpus=4, num_gpus=0)

print(f'Ray is {"up" if ray.is_initialized() else "down"}')

2022-02-11 15:54:16,382	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8266


Ray is up


Note: If you need to debug your Ray program, it may be easier to do everything on a single process. You can force all Ray functions to occur on a single process by enabling local mode: `ray.init(local_mode=True)`

In [19]:
# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def f(x):
    return x * x

# our function has a remote method now...
futures = [f.remote(i) for i in range(4)]
print(ray.get(futures)) # [0, 1, 4, 9]

[0, 1, 4, 9]


In [17]:
ray.shutdown()
print(f'Ray is {"up" if ray.is_initialized() else "down"}')

Ray is down


Ray Actors: An actor is essentially a stateful worker (or a service). 

In [1]:
import ray

if not ray.is_initialized():
    ray.init(num_cpus=4)

@ray.remote
class Bears:
    
    def __init__(self, name):
        self.name = name
        
    def growl(self):
        return(f"{self.name} growls.")

bears_list = [Bears.remote("Oski"), Bears.remote("Yogi")]
futures = [bear.growl.remote() for bear in bears_list]
print(ray.get(futures))

2022-02-11 16:20:57,302	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


['Oski growls.', 'Yogi growls.']


Asyncio for Actors: https://docs.ray.io/en/latest/async_api.html#async-actors

In [2]:
from ray.util import ActorPool

@ray.remote
class Actor:

    def square_it(self, n):
        return n ** 2

a1, a2 = Actor.remote(), Actor.remote()
pool = ActorPool([a1, a2])

# pool.map(..) returns a Python generator object ActorPool.map
gen = pool.map(lambda a, v: a.square_it.remote(v), [1, 2, 3, 4])
print([v for v in gen])

[1, 4, 9, 16]
